# Yelp API Webscraping

In [17]:
import pandas as pd 
import json
import requests
import time

# Compiled Code
- Don't forget to comment so we can understand!

## known issues:
- function populates dataframe with NaN rows
- do we want business name?
- businesses have multiple categories and function only picks up one at a time
    - currently the 'alias' portion picks up the alias of the business (name), not the category alias

In [7]:
# NOTHING IN THIS CELL NEEDS TO GET CHANGED 
# IF YOU HAVE YOUR API KEY IN 'creds.json' IN ./Assets

# format your json file as a dictionary containing api key with DOUBLE QUOTES
# {"api": "your_super_long_api_key"}
creds_file = open('../Assets/creds.json')

# load credentials into variable
yelp_credentials = json.loads(creds_file.read())
api_key = yelp_credentials['api']
headers = {'Authorization': 'Bearer %s' % api_key}

# this is the url we use to make broad business searches
# https://www.yelp.com/developers/documentation/v3/business_search

url = 'https://api.yelp.com/v3/businesses/search'

In [8]:
# depending on what we want to search we can change values in this dictionary
params = {'term':'food', 'location': 'Los Angeles'}

In [9]:
# ONLY RUN THIS CELL IF YOU WANT TO MAKE A REQUEST
req = requests.get(url, params=params, headers=headers)
print(f'Status Code: {req.status_code}')

Status Code: 200


In [10]:
yelp = json.loads(req.text)

In [11]:
businesses_object = yelp['businesses']
businesses_object[15]

# id
# coordinates
# price
# review_count
# rating
# location > zip_code
# location > city
# categories > alias

{'id': 'CbW8U0QAwh5XRkaLt0xNZA',
 'alias': 'masa-of-echo-park-los-angeles',
 'name': 'Masa of Echo Park',
 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/k8LDtpavOtjYyjgHcErZaA/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/masa-of-echo-park-los-angeles?adjust_creative=1hBkN0UhEY8hcT3qXV3ivQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=1hBkN0UhEY8hcT3qXV3ivQ',
 'review_count': 4023,
 'categories': [{'alias': 'newamerican', 'title': 'American (New)'},
  {'alias': 'pizza', 'title': 'Pizza'},
  {'alias': 'salad', 'title': 'Salad'}],
 'rating': 4.0,
 'coordinates': {'latitude': 34.077585, 'longitude': -118.259599},
 'transactions': ['pickup'],
 'price': '$$',
 'location': {'address1': '1800 W Sunset Blvd',
  'address2': None,
  'address3': '',
  'city': 'Los Angeles',
  'zip_code': '90026',
  'country': 'US',
  'state': 'CA',
  'display_address': ['1800 W Sunset Blvd', 'Los Angeles, CA 90026']},
 'phone': '+12139891558',
 'display_phone': '(2

In [26]:
# stole this function from project 3 - jerry

def get_price(yelp_object):
    businesses = []
    # empty list container containing dictionaries representing each unique business
    
    businesses_object = yelp_object['businesses']
    
    # looking at each subreddit which is passed into our function
    for i, business in enumerate(list(businesses_object)):
        
        # each dictionary will contain all of the desired information from each post
        business_dict = {}

        if 'price' in businesses_object[i].keys():
            business_dict['id']           = business['id']
            business_dict['latitude']     = business['coordinates']['latitude']
            business_dict['longitude']    = business['coordinates']['longitude']
            business_dict['price']        = business['price']
            business_dict['review_count'] = business['review_count']
            business_dict['rating']       = business['rating']
            business_dict['zip_code']     = business['location']['zip_code']
            business_dict['city']         = business['location']['city']
            business_dict['alias']        = business['alias']

        # populate the posts list with each post dictionary
        businesses.append(business_dict)

        # just a little sanity check to see how far along our function is going
        #print(f'{i+1} out of {len(businesses_object)}')
    return businesses
    #businesses = pd.DataFrame(businesses)
   
    

list

In [52]:
business_list = []
while True:
    business_list.extend(get_price(yelp))
    time.sleep(1)

KeyboardInterrupt: 

In [53]:
len(business_list)

6100

In [54]:
df = pd.DataFrame(business_list)

In [56]:
df

,id,latitude,longitude,price,review_count,rating,zip_code,city,alias
0,CcqraT0cuGKYEcZ1ri_kxg,34.040403,-118.253512,$$,689.0,5.0,90015,Los Angeles,broken-mouth-lees-homestyle-los-angeles-5
1,KQBGm5G8IDkE8LeNY45mbA,34.045605,-118.236061,$$,8205.0,4.0,90013,Los Angeles,wurstküche-los-angeles-2
2,pjh40JY5YwWeV8aKhkXERg,34.065770,-118.308470,$,702.0,4.5,90020,Los Angeles,myungrang-hot-dog-california-market-la-los-ang...
3,b4SH4SbQUJfXxh6hNkF0wg,34.050529,-118.248619,$,5597.0,4.0,90013,Los Angeles,eggslut-los-angeles-7
4,sYn3SNQP-j2t2XSwjlCbRg,34.064690,-118.308760,$$,1618.0,4.5,90020,Los Angeles,montys-good-burger-los-angeles
...,...,...,...,...,...,...,...,...,...
6095,CbW8U0QAwh5XRkaLt0xNZA,34.077585,-118.259599,$$,4023.0,4.0,90026,Los Angeles,masa-of-echo-park-los-angeles
6096,fjWpZPr4YDJRKdnj0fqlkg,34.083320,-118.327380,$$,33.0,5.0,90038,Los Angeles,omakase-teriyaki-los-angeles
6097,ohosmz6FXVAeoW5nUkYwng,34.090596,-118.277172,$$,2151.0,4.5,90026,Los Angeles,pine-and-crane-los-angeles
6098,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
df.to_csv('../data/echo1023.csv')

## aerika section

In [1]:
hi

NameError: name 'hi' is not defined

In [2]:
my

NameError: name 'my' is not defined

In [3]:
name

NameError: name 'name' is not defined

## echo section

added comment

In [ ]:
added cell

## jerry section

In [ ]:
# ds;alkfjds;lfkas;dghas'dlfjas;dlkfjas'dfj

In [1]:
# this is another change a;sdfja;dfkasd;fajsd;fasdkfjas;fwef

In [2]:
# yet another change ;asdlfkjas;dknva;djaweoifsd;laknfas;dfknasd

In [3]:
x = np.exp(52)
x

3.831008000716577e+22